In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

# read "mpd_stats.csv" with relative path
mpd_stats = pd.read_csv('Input_Data/mpd_stats.csv', skiprows=2, header = 1)


In [2]:
# convert the "Date" column to datetime
mpd_stats["idt"] = pd.to_datetime(mpd_stats["idt"])

#rename the idt column to Date
mpd_stats.rename(columns = {"idt":"Date"}, inplace = True)

In [3]:
mpd_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12249 entries, 0 to 12248
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   market           12249 non-null  object        
 1   Date             12249 non-null  datetime64[ns]
 2   maturity_target  11294 non-null  float64       
 3   mu               12245 non-null  float64       
 4   sd               12245 non-null  float64       
 5   skew             12245 non-null  float64       
 6   kurt             12245 non-null  float64       
 7   p10              11345 non-null  float64       
 8   p50              12249 non-null  float64       
 9   p90              12249 non-null  float64       
 10  lg_change_decr   12230 non-null  float64       
 11  prDec            12249 non-null  float64       
 12  lg_change_incr   12230 non-null  float64       
 13  prInc            11988 non-null  float64       
dtypes: datetime64[ns](1), float64(12), obj

In [4]:
mpd_stats.shape

(12249, 14)

In [5]:
# check for missing values
mpd_stats.isnull().sum()

market               0
Date                 0
maturity_target    955
mu                   4
sd                   4
skew                 4
kurt                 4
p10                904
p50                  0
p90                  0
lg_change_decr      19
prDec                0
lg_change_incr      19
prInc              261
dtype: int64

In [6]:
# # Filter rows where 'Date' is between 2010 and 2023
# filtered_mpd_stats = mpd_stats[(mpd_stats['Date'].dt.year >= 2012) & (mpd_stats['Date'].dt.year <= 2023)].copy()
# Filtering the dataset to include only the records from the year 2012 onwards
filtered_mpd_stats = mpd_stats[(mpd_stats['Date'].dt.year >= 2012)].copy()

# Extract year from Date for easier analysis
filtered_mpd_stats['Year'] = filtered_mpd_stats['Date'].dt.year

# Identify markets that have data for each year in the range 2013 to 2023
years = range(2012, 2024)  # 2023 is inclusive
markets_with_complete_data = filtered_mpd_stats.groupby('market').filter(lambda x: all(y in x['Year'].values for y in years))['market'].unique()

# Filter the original dataframe to keep only the markets identified
mpd_stats_filtered_complete = mpd_stats[mpd_stats['market'].isin(markets_with_complete_data)]

# # remove rows that date is before 2013 jan 10
mpd_stats_filtered_complete = mpd_stats_filtered_complete[mpd_stats_filtered_complete['Date'] >= '2013-01-10']

print("Number of Unique market variables: ", len(mpd_stats_filtered_complete['market'].unique()))
print("Listed Market Variables: ", mpd_stats_filtered_complete['market'].unique())

Number of Unique market variables:  14
Listed Market Variables:  ['bac' 'citi' 'corn' 'euro' 'gold' 'infl5y' 'iyr' 'pound' 'silver'
 'soybns' 'sp12m' 'sp6m' 'wheat' 'yen']


In [7]:
# print the staring and ending date for each market in mpd_stats_filtered_complete
start_end_dates = mpd_stats_filtered_complete.groupby('market').agg(start_date=('Date', 'min'), end_date=('Date', 'max'))
print(start_end_dates)

       start_date   end_date
market                      
bac    2013-01-10 2024-02-07
citi   2013-01-10 2024-02-07
corn   2013-01-10 2024-02-07
euro   2013-01-10 2024-02-07
gold   2013-01-10 2024-02-07
infl5y 2013-01-10 2024-02-07
iyr    2013-01-10 2024-02-07
pound  2013-01-10 2024-02-07
silver 2013-01-10 2024-02-07
soybns 2013-01-10 2024-02-07
sp12m  2013-01-10 2024-02-07
sp6m   2013-01-10 2024-02-07
wheat  2013-01-10 2024-02-07
yen    2013-01-10 2024-02-07


Upsampling for weekly data by forward filling, as majority of the market data frequency are in weekly basis

In [8]:
mpd_stats_filtered_complete.set_index('Date', inplace=True)

In [9]:
mpd_stats_filtered_complete.shape

(7627, 13)

In [10]:
# Ensure 'Date' is the DataFrame index
# mpd_stats_filtered_complete.set_index('Date', inplace=True)

def resample_and_fill(group):
    # Define the cutoff date
    cutoff_date = '2014-09-04'
    
    # Filter the group for dates before the cutoff
    group_before_cutoff = group[:cutoff_date]
    
    # Resample to weekly frequency and forward fill
    group_resampled = group_before_cutoff.resample('W-Thu').ffill()
    
    # Concatenate with the part of the group after the cutoff date
    # group_after_cutoff = group[cutoff_date:]
    # Adjust the slicing to exclude the cutoff date from the second part
    group_after_cutoff = group[pd.to_datetime(cutoff_date) + pd.Timedelta(days=1):]
    result = pd.concat([group_resampled, group_after_cutoff])
    
    return result

# Apply the function to each group and combine the results
mpd_stats_weekly = mpd_stats_filtered_complete.groupby('market', group_keys=False).apply(resample_and_fill)

# Reset the index if necessary
mpd_stats_weekly.reset_index(inplace=True)

In [11]:
mpd_stats_weekly.shape

(8081, 14)

In [12]:
mpd_stats_weekly.fillna(method='ffill', inplace=True)
nan_rows = mpd_stats_weekly.isnull().sum()
print(nan_rows)

Date               0
market             0
maturity_target    0
mu                 0
sd                 0
skew               0
kurt               0
p10                0
p50                0
p90                0
lg_change_decr     0
prDec              0
lg_change_incr     0
prInc              0
dtype: int64


In [13]:
# check the number of rows for each market
mpd_stats_weekly.groupby('market').size()

market
bac       578
citi      578
corn      578
euro      578
gold      578
infl5y    571
iyr       577
pound     578
silver    575
soybns    578
sp12m     578
sp6m      578
wheat     578
yen       578
dtype: int64

### Feature Engineering: need to do some layout modification for regression

In [14]:
mpd_stats_weekly['market'].unique()

array(['bac', 'citi', 'corn', 'euro', 'gold', 'infl5y', 'iyr', 'pound',
       'silver', 'soybns', 'sp12m', 'sp6m', 'wheat', 'yen'], dtype=object)

In [15]:
mpd_stats_weekly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8081 entries, 0 to 8080
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             8081 non-null   datetime64[ns]
 1   market           8081 non-null   object        
 2   maturity_target  8081 non-null   float64       
 3   mu               8081 non-null   float64       
 4   sd               8081 non-null   float64       
 5   skew             8081 non-null   float64       
 6   kurt             8081 non-null   float64       
 7   p10              8081 non-null   float64       
 8   p50              8081 non-null   float64       
 9   p90              8081 non-null   float64       
 10  lg_change_decr   8081 non-null   float64       
 11  prDec            8081 non-null   float64       
 12  lg_change_incr   8081 non-null   float64       
 13  prInc            8081 non-null   float64       
dtypes: datetime64[ns](1), float64(12), objec

In [16]:
# create a df for each market with column name of market
features = pd.DataFrame(mpd_stats_weekly['market'].unique(), columns=['market'])
# add a column for with the name of feature_id
features['feature_name'] = mpd_stats_weekly['market'].unique()
# rename the market column to market category
features.rename(columns = {"market":"market_category"}, inplace = True)



In [17]:
# if the feature_id has "bac" 'citi', then change market_category to "Bank"
features.loc[features['feature_name'].isin(['bac', 'citi']), 'market_category'] = 'Bank'
# if the feature_id has "silver", "corn", "soybns", "gold", 'iyr', 'wheat', then change market_category to "Commodity"
features.loc[features['feature_name'].isin(['silver', 'corn', 'soybns', 'gold', 'iyr', 'wheat']), 'market_category'] = 'Commodity'
# if the feature_id has 'yen', 'euro', 'pound', then change market_category to "Currency"
features.loc[features['feature_name'].isin(['yen', 'euro', 'pound']), 'market_category'] = 'Currency'
# if the feature_id has 'sp12m', 'sp6m', then change market_category to "Equity"
features.loc[features['feature_name'].isin(['sp12m', 'sp6m']), 'market_category'] = 'Equity'
# if the feature_id has 'infl1y',  'infl2y', 'infl5y', then change market_category to "Inflation"
features.loc[features['feature_name'].isin(['infl1y', 'infl2y', 'infl5y']), 'market_category'] = 'Inflation'
# if the feature_id has 'tr10yr', 'tr5yr', 'LR3y3m','LR5y3m' then change market_category to "Rates"
features.loc[features['feature_name'].isin(['tr10yr', 'tr5yr', 'LR3y3m','LR5y3m']), 'market_category'] = 'Rates'


In [18]:
# group the features by market_category
features = features.sort_values(by='feature_name')
# add id number for each feature
features['id'] = range(1, len(features) + 1)
features

,market_category,feature_name,id
0,Bank,bac,1
1,Bank,citi,2
2,Commodity,corn,3
3,Currency,euro,4
4,Commodity,gold,5
5,Inflation,infl5y,6
6,Commodity,iyr,7
7,Currency,pound,8
8,Commodity,silver,9
9,Commodity,soybns,10


In [19]:
# Create a dictionary to map market names to feature IDs
market_to_id = features.set_index('feature_name')['id'].to_dict()

# Initialize df_feature_engineered with the Date column
df_feature_engineered = pd.DataFrame(mpd_stats_weekly[mpd_stats_weekly['market'] == 'bac']['Date'].copy()).drop_duplicates()

# Iterate over each market to merge its data into df_feature_engineered
for market in mpd_stats_weekly['market'].unique():
    # Select the data for the current market, excluding the 'market' column
    market_data = mpd_stats_weekly[mpd_stats_weekly['market'] == market].drop(columns=['market']).copy()
    
    # Get the feature ID for the current market
    feature_id = market_to_id[market]
    
    # Rename the columns based on the feature ID, except for 'Date'
    market_data.rename(columns={col: f'f{feature_id}_{col}' for col in market_data.columns if col != 'Date'}, inplace=True)
    
    # Merge the data into df_feature_engineered
    df_feature_engineered = df_feature_engineered.merge(market_data, on='Date', how='left')

# This ensures that the 'market' column is not included in the final DataFrame

In [20]:
df_feature_engineered.shape

(578, 169)

In [21]:
# find nan values or missing values
nan_rows = df_feature_engineered.isnull().sum()
# To show full output without truncation
pd.set_option('display.max_rows', 50)
print(nan_rows)

Date                  0
f1_maturity_target    0
f1_mu                 0
f1_sd                 0
f1_skew               0
                     ..
f14_p90               1
f14_lg_change_decr    1
f14_prDec             1
f14_lg_change_incr    1
f14_prInc             1
Length: 169, dtype: int64


Fill Missing Value: Interpolation: This method is useful for time series data. It fills missing values time-weighted interpolation, based on the index, avioding future looking bias

In [22]:
# Interpolate missing values linearly without using future data
df_feature_engineered['Date'] = pd.to_datetime(df_feature_engineered['Date'])
df_feature_engineered.set_index('Date', inplace=True)
df_feature_engineered.interpolate(method='time', inplace=True)

In [23]:
# reset index
df_feature_engineered.reset_index(inplace=True)
# to csv
df_feature_engineered.to_csv('Output_Data/cleaned_feature_engineered_mpd.csv', index=False)